### Config

In [1]:
from pathlib import Path

In [30]:
# Path specifications
PROJECT_NAME = 'costa_rica_poverty'
PACKAGE_ROOT = Path().resolve().parent
INPUT_DIR = f"{PACKAGE_ROOT}/{PROJECT_NAME}/packages/model/model/input/data"
ASSET_DIR = f"{PACKAGE_ROOT}/{PROJECT_NAME}/packages/model/model/assets"
OUTPUT_DIR = f"{PACKAGE_ROOT}/{PROJECT_NAME}/packages/model/model/output"

# Data file specifications
CODEBOOK = 'codebook.csv'
DATA_FILE = 'train.csv'

### Data cleaning: test drafting

In [3]:
import pandas as pd
import numpy as np

In [4]:
codebook = pd.read_csv(f"{ASSET_DIR}/{CODEBOOK}")
df = pd.read_csv(f"{INPUT_DIR}/{DATA_FILE}")
print(df.shape)

(9557, 143)


In [ ]:
################## Head of households

#### TEST
def test_head_of_household_exists(raw_data):
    # Given
    hh_head = df.groupby('idhogar')['parentesco1'].sum().reset_index()
    
    # When
    hh_head_none = hh_head[hh_head['parentesco1']!=1]
    
    # Then
    assert hh_head_none.shape[0]==0
    
#### FIX
class HeadOfHouseholdExist(BaseEstimator, TransformerMixin):
    def __init__(self, variables: List[str]) -> None:
        '''
        Specify variable input as list of strings representing column names of a pd.DataFrame.
        '''
        # YOUR CODE HERE
        return None
    
    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        '''
        Required method for Sklearn TransformerMixin class.
        Remains inactive and performs no action for now.
        Leave as is.
        '''
        return self

    def transform(self, X: pd.DataFrame, y: pd.Series = None):
        '''
        Creates a copy of input dataframe, which is passed to method via the sklearn Pipeline.
        This method performs changes to the dataframe with reference to specified variables.
        The modified copy of the original dataframe is then returned and passed to the next step in pipeline.
        Define your specific transformation here.
        '''
        X = X.copy()
        # YOUR CODE HERE
        hh_head = df.groupby('idhogar')['parentesco1'].sum().reset_index()
        hh_head_none= hh_head[hh_head['parentesco1']!=1]
        hh_head_none
        
        X = X[~X['idhogar'].isin(hh_head_none['idhogar'].values)]

        return X

#### TEST
def test_household_target_match(raw_data):
    # Given
    householdTargetsMatching = (df.groupby('idhogar')['Target'].nunique()==1).reset_index()
    
    # When
    householdTargetMisMatch = householdTargetsMatching[householdTargetsMatching['Target']==False]
    
    # Then
    assert householdTargetMisMatch.shape[0]==0

    
#### FIX
class HouseholdTargetsMatch(BaseEstimator, TransformerMixin):
    def __init__(self, variables: List[str]) -> None:
        '''
        Specify variable input as list of strings representing column names of a pd.DataFrame.
        '''
        # YOUR CODE HERE
        return None
    
    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        '''
        Required method for Sklearn TransformerMixin class.
        Remains inactive and performs no action for now.
        Leave as is.
        '''
        return self

    def transform(self, X: pd.DataFrame, y: pd.Series = None):
        '''
        Creates a copy of input dataframe, which is passed to method via the sklearn Pipeline.
        This method performs changes to the dataframe with reference to specified variables.
        The modified copy of the original dataframe is then returned and passed to the next step in pipeline.
        Define your specific transformation here.
        '''
        X = X.copy()
        # Identify household target mismatches
        householdTargetsMatching = (X.groupby('idhogar')['Target'].nunique()==1).reset_index()
        householdTargetMisMatch = householdTargetsMatching[householdTargetsMatching['Target']==False]
        
        # set all member targets to head of household target
        for household in householdTargetMisMatch['idhogar']:
            correct_poverty_level = int(X[(X['idhogar'] == household) & (X['parentesco1'] == 1)]['Target'])
            X.loc[df['idhogar'] == household, 'Target'] = correct_poverty_level

        return X


In [15]:
hh_head = df.groupby('idhogar')['parentesco1'].sum().reset_index()
hh_head_none = hh_head[hh_head['parentesco1']!=1]
hh_head_none
df = df[~df['idhogar'].isin(hh_head_none['idhogar'].values)]

,idhogar,parentesco1
38,03c6bdf85,0
114,09b195e7a,0
230,1367ab31d,0
331,1bc617b23,0
645,374ca5a19,0
1143,61c10e099,0
1268,6b1b2405f,0
1613,896fe6d3e,0
1867,a0812ef17,0
2027,ad687ad89,0


In [47]:
len(os.listdir(OUTPUT_DIR))>1

True

In [ ]:
class MissingValuesEvaluator:
    pass

class VariableDistributionEvaluator:
    pass